## Create Kubernetes Cluster on Azure or GKE

This notebook can be used to launch a Kubernetes Cluster on [Azure](https://portal.azure.com) or the Google Container Engine (GKE) for Kubernetes.  It is designed to be run via the included Docker container, but it can be run locally if the appropriate tools are installed.  

If you are just getting started run the command below to create your sample config.  Because config.yaml is listed in the .gitignore, it won't be committed. (It may have details about your specific password for implementations).


In [ ]:
!cp ../config/config.sample  ../config/config.yaml

Now go ahead and comment out the cell above so you don't accidentally overwrite your work. 
```
#!cp ../../config/config.sample  ../../config/config.yaml
```

### Load the Config File and Helpers 
Set the `config` file and the `path` and then run the code below.  This will load some helper files. 

In [15]:
import sys 
import importlib
import ruamel.yaml
config_file='config/config.yaml'  #The configuration value set above.
path='/home/jovyan/work/admin-tools/' #Update this value if running locally.
sys.path.append(path+"lib/kuberutils") #Adds a local library path.
import kuberutils as ku   #This imports some utilities.
importlib.reload(ku)      #This just reloads the utilities.
cf=ku.initialize(path,config_file)  #This initializes a configuration object.

### Print Configuration
Optionally you can print the configuration and common commands for your desired cluster. You can use this as a reference and copy and paste into the terminal.

In [9]:
#This will print common commands for your clustl
print(ruamel.yaml.dump(cf, sys.stdout, Dumper=ruamel.yaml.RoundTripDumper))

#Where are you running notebooks? This is used for path.
docker: true     #True if executing on the Kuberlytics Docker Stack.
docker_path: /home/jovyan/work/admin-tools      #This should not change.
local_path: /Users/jasonkuruzovich/githubdesktop/0_class/admin-tools #This is the local path
cloud_provider: azure   #google or azure
#Start of Google Specific Configuration
g_project: kuberlytics             #Google project name
g_cluster_name: kuberlytics        #Name of your Google Cluster
g_region: us-east1                 #Selection from gcloud compute regions list.
g_zone: us-east1-b                  #Selection from gcloud compute regions list.
g_machine_type: n1-highmem-4        #Type of Server
g_account: jkuruzovich@gmail.com    #Email Associated with the account.
g_authorization_file: auth.json     #Service account authorization file.
g_service_account_name: kuberlytics2  #Service account name.
g_fixedip_namespace: jupyterhub-dojo
g_num_nodes: 1                      #The default nu

### Web Login

In order to use the web login, you need to run the command below and then enter the code in the generated web link.  Careful not to commit this code to github repository. 

TBD: Need someone to research loging in with service account. The way google works you can download a json file that can be used for authentication.  This is better for eventual automation. 


In [3]:
#For Azure: Use this. 
!az login


To sign in, use a web browser to open the page https://aka.ms/devicelogin and enter the code AZCFX6HGN to authenticate.
[
  {
    "cloudName": "AzureCloud",
    "id": "652b3848-14d2-4276-af14-fbcd7db53805",
    "isDefault": true,
    "name": "Microsoft Azure Sponsorship",
    "state": "Enabled",
    "tenantId": "aaf653e1-0bcd-4c2c-8658-12eb03e15774",
    "user": {
      "name": "kuruzj@rpi.edu",
      "type": "user"
    }
  }
]


### Google - Service Account Login

This requires a authentication file for a service account to be placed in /config/gcloud.  


In [ ]:
#Login 
#ku.bash_command('login',cf) #tbd

### Create Project 
Google calls them projects.  Azure calles them resource groups. Either way you need one. This useful to track spending and also ensure you delete all resources at the end. 


In [10]:
#This will set the create the project. 
result= ku.bash_command('create_project',cf)

#TBD, write a test to see if State= "Succeeded"

Executing create_project:
 az group create --name=kubwestus2rpi --location=westus2


In [11]:
#This will set the project (Google Only). 
ku.bash_command('set_project',cf)

Executing set_project:
 echo 'No need to set project in Azure.'


'No need to set project in Azure.\n'

In [12]:
#This will set the zone (Google Only). 
ku.bash_command('set_zone',cf)


Executing set_zone:
 echo 'No need to set project in Azure.'


'No need to set project in Azure.\n'

### Create the Cluster
This will create your Kubernetes Cluster. You have to wait for about 5 minutes before this finishes creating.

#TBD, write a test to see if State= "Succeeded"


In [13]:
#This will create the cluseter and can be very long running. 
print(ku.bash_command('create_cluster', cf))


Executing create_cluster:
 az acs create --orchestrator-type=kubernetes --resource-group=kubwestus2rpi --name=kubwestus2rpi --dns-prefix=kubwestus2rpi --agent-count=1 --agent-vm-size=Standard_D3_v2 --generate-ssh-keys --no-wait
AAD role propagation done[############################################]  100.0000%


### Get Credentials for Kubectl
We need to add the credentials for Kubectl to work. You need a bit of time for your Kubernetes to launch.

In [16]:
#gcloud container clusters get-credentials kuberlytics
print(ku.bash_command('get_credentials',cf))


Executing get_credentials:
 az acs kubernetes get-credentials --resource-group=kubwestus2rpi --name=kubwestus2rpi
ERROR: Could not find container service resource: kubwestus2rpi in subscription: 652b3848-14d2-4276-af14-fbcd7db53805, resourceGroup: kubwestus2rpi.



In [ ]:
#Check to see if we have Kubectl working. 
print(ku.bash_command("kubectl cluster-info"))


In [ ]:
#Check notes with Kubectl
print(ku.bash_command("kubectl get node"))


### Helm Installation.  
We are going to be utilizing Helm for  installations of a variety of analytics tools.  This command will install Tiller on your cluster.  As they say, "Happy Helming!" 

A critical factor for Helm is that you have the same version running locally and via your machine.  If you run helm version and you have the right version, then you should be fine.

```
Client: &version.Version{SemVer:"v2.6.2", GitCommit:"be3ae4ea91b2960be98c07e8f73754e67e87963c", GitTreeState:"clean"}
Server: &version.Version{SemVer:"v2.6.2", GitCommit:"be3ae4ea91b2960be98c07e8f73754e67e87963c", GitTreeState:"clean"}
```

To install the appropriate version: 

```
curl https://raw.githubusercontent.com/kubernetes/helm/master/scripts/get > get_helm.sh
chmod 700 get_helm.sh
RUN get_helm.sh --version v2.6.2

```


In [51]:
#This may need to be run more than once if you get a "cannot connect to server."
result=ku.bash_command("helm version")
result

Executing helm version:
 helm version


'Client: &version.Version{SemVer:"v2.6.2", GitCommit:"be3ae4ea91b2960be98c07e8f73754e67e87963c", GitTreeState:"clean"}\nServer: &version.Version{SemVer:"v2.6.2", GitCommit:"be3ae4ea91b2960be98c07e8f73754e67e87963c", GitTreeState:"clean"}\n'

In [ ]:
# Both ACS and GKE install tiller on the server.
print(ku.bash_command('install_helm',cf))

In [49]:
print(ku.bash_command("helm init --upgrade"))

Executing helm init --upgrade:
 helm init --upgrade
Creating /home/jovyan/.helm 
Creating /home/jovyan/.helm/repository 
Creating /home/jovyan/.helm/repository/cache 
Creating /home/jovyan/.helm/repository/local 
Creating /home/jovyan/.helm/plugins 
Creating /home/jovyan/.helm/starters 
Creating /home/jovyan/.helm/cache/archive 
Creating /home/jovyan/.helm/repository/repositories.yaml 
$HELM_HOME has been configured at /home/jovyan/.helm.

Tiller (the Helm server-side component) has been upgraded to the current version.
Happy Helming!



### Enabling Autoscaling (optional)

This should launch a pod within your kubernetes cluster that will handle autoscaling of the cluster. Note that this seems to take a while and may even timeout. Consider opening and running in a terminal session. 

In [ ]:
#Google only. Currently Not possible with azure
ku.bash_command(cf['autoscale'])

#### That is it! You now have your own Kubernetes cluster that is ready to go. 

### Resize a Cluster
To stop a cluster without deleting it you just resize it to 0.

In [ ]:
#TBD
ku.bash_command(cf['class_size_cluster'])

In [ ]:
#TBD
#ku.bash_command(cf_g['stop_cluster'])

In [ ]:
#TBD
ku.bash_command(cf['normal_size_cluster'])

### Deleting a Kubernetes Cluster

This will delete the Kubernetes cluster by deleting the entire project. This will prefent any future charges. 

In [17]:
#Always delete the namespace first. 
print(ku.bash_command('delete_project',cf))

Executing delete_project:
 az group delete --name=kubwestus2rpi --yes --no-wait

